In [1]:
import pandas as pd
import numpy as np
import re
from kiwipiepy import Kiwi
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt


In [2]:
data = pd.read_csv('weaviate_data(임시).csv')
data.head(2)

,recipeID,recipe_name,summary,ingredient_name,full_step,category,image_link
0,0,우엉양배추볶음밥,찬밥이 기다려지는 우엉양배추볶음밥입니다.,[재료] 찬 밥 적당량 양배추 적당량 우엉 적당량 느타리버섯 조금 다시마 ...,찬밥을 준비해주세요. 양배추와 우엉을 준비해주시고 양배추는 깨끗히 씻어서 잘게 잘라...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...
1,1,아삭아삭 연근장아찌 만들기,연근은 비타민C와 식이섬유소가 풍부하고 빈혈예방에 탁월하고 코피가 자주나는 아이들에...,[재료] 연근 [양념비율(200ml 계량컵 기준)] 간장 1 물 1 설탕 0...,연근은 감자칼로 껍질을 벗겨주었어요. 슥슥 0.5mm정도 잘라주었답니다. 찬물에 식...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874 entries, 0 to 873
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   recipeID         874 non-null    int64 
 1   recipe_name      874 non-null    object
 2   summary          835 non-null    object
 3   ingredient_name  842 non-null    object
 4   full_step        840 non-null    object
 5   category         874 non-null    object
 6   image_link       874 non-null    object
dtypes: int64(1), object(6)
memory usage: 47.9+ KB


In [4]:
source = []
for ingredient_name in data['ingredient_name']:
    ingredient_name_str = str(ingredient_name)
    if '양념' in ingredient_name_str:
        ingredient_matches = re.findall(r'\[.*?\](.*?)\양념', ingredient_name_str)
        source_per_ingredient = []
        for ingredient_match in ingredient_matches:
            ingredient_cleaned = re.sub(r'[0-9]+[가-힣]+', '', ingredient_match).strip().split()
            for ing in ingredient_cleaned:
                if ing:
                    ingredient_final = re.sub(r'[^가-힣\s]', '', ing).strip()
                    if ingredient_final:
                        source_per_ingredient.extend(ingredient_final.split())
    else:
        ingredient_matches = re.findall(r'\[.*?\](.*)', ingredient_name_str)
        source_per_ingredient = []
        for ingredient_match in ingredient_matches:
            ingredient_cleaned = re.sub(r'[0-9]+[가-힣]+', '', ingredient_match).strip().split()
            for ing in ingredient_cleaned:
                if ing:
                    ingredient_final = re.sub(r'[^가-힣\s]', '', ing).strip()
                    if ingredient_final:
                        source_per_ingredient.extend(ingredient_final.split())
    # if source_per_ingredient:
    source.append(source_per_ingredient)

In [5]:
len(source)

874

In [6]:
sum(1 for item in source if not item)

32

In [7]:
data['main_source'] = source

In [8]:
data_re = pd.read_csv('user_review.csv')

In [9]:
data_re.head(3)

,recipe_name,user_id,posting_time,rating,review_content,category,recipeID,reviewID
0,아삭아삭 연근장아찌 만들기,ronaldo115,2017-06-08 17:52:21,5,연근조림 정말 맛있어요!\n꼭 다이어트해서 설현 몸매가 될거에요 ㅎㅎㅎ,저칼로리 다이어트,1,3341
1,아삭아삭 연근장아찌 만들기,23990030,2018-09-18 23:58:50,5,꿀맛ㅇ,저칼로리 다이어트,1,472
2,아삭아삭 연근장아찌 만들기,66191572,2016-08-24 18:06:15,1,맛있어욤 다름 다이어트 음식도 많이 알려주세요!♡♡♡\n다름이아니라 이 음식을 먹고...,저칼로리 다이어트,1,1953


In [10]:
data.head(1)

,recipeID,recipe_name,summary,ingredient_name,full_step,category,image_link,main_source
0,0,우엉양배추볶음밥,찬밥이 기다려지는 우엉양배추볶음밥입니다.,[재료] 찬 밥 적당량 양배추 적당량 우엉 적당량 느타리버섯 조금 다시마 ...,찬밥을 준비해주세요. 양배추와 우엉을 준비해주시고 양배추는 깨끗히 씻어서 잘게 잘라...,저칼로리 다이어트,https://recipe1.ezmember.co.kr/cache/recipe/20...,"[찬, 밥, 적당량, 양배추, 적당량, 우엉, 적당량, 느타리버섯, 조금, 다시마,..."


In [11]:
df = pd.merge(data, data_re, on=['recipeID', 'recipe_name', 'category'], how='outer')

In [12]:
kiwi = Kiwi()
stop_words = ['물', '약간', '적당', '적당량', '조금', '때', '레시피', '요리', '저', '거', '것', '수', '나']

def extract_keywords(text):
    stop_words = ['물', '약간', '적당', '적당량', '조금', '때', '레시피', '요리', '저', '거', '것', '수', '나']
    keywords = []
    text = text.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    for title in text:
        titles = []
        result = kiwi.analyze(title)
        for token, tag, _ , _ in result[0][0]:
            if tag[0] in 'NN':
                if token not in stop_words:
                    titles.append(token)
        keywords.append(titles)
    return keywords

In [13]:
name = extract_keywords(df['recipe_name'].apply(str))
summary = extract_keywords(df['summary'].apply(str))
source = extract_keywords(df['main_source'].apply(str))

In [14]:
review = extract_keywords(df['review_content'].apply(str))


In [15]:
list_all = []

for item_1, item_2, item_3, item_4 in zip(name, summary, df['main_source'], review):
    combined_list = item_1 + item_2 + item_3 + item_4
    list_all.append(combined_list)

In [16]:
df['all'] = list_all

In [17]:
all_keyword = []
for i in df['all']:
    for item in i:
        all_keyword.append(item)
print(len(all_keyword))

91541


In [18]:
unique_keyword = set(all_keyword)
# print(unique_keyword)
print(len(unique_keyword))


4048


In [19]:
'우엉' in unique_keyword

True

In [20]:
total_recipe_count = len(set(data['recipe_name']))
keyword_count_dict = dict.fromkeys(unique_keyword)

for keyword_list in df['all']:
    for keyword in keyword_list:
        if keyword_count_dict[keyword.strip()] == None:
            keyword_count_dict[keyword.strip()] = 1
        else:
            keyword_count_dict[keyword.strip()] += 1

In [21]:
keyword_idf = dict()
for each_keyword in keyword_count_dict:
    keyword_idf[each_keyword] = np.log10(total_recipe_count/keyword_count_dict[each_keyword])

In [22]:
sorted_data_dict = dict(sorted(keyword_idf.items(), key=lambda item: item[1], reverse=True))

In [23]:
all_category = data['category'].explode().tolist()
all_keyword = df['all'].explode().tolist()
unique_keyword = set(all_keyword)
unique_category = set(all_category)

In [24]:
unique_category

{'간 건강',
 '골다공증',
 '노화방지',
 '머슬업 다이어트',
 '변비',
 '스트레스 해소',
 '암 예방',
 '위 건강',
 '장 건강',
 '저칼로리 다이어트',
 '치매예방',
 '피로회복',
 '혈당조절',
 '혈액순환',
 '호흡기 건강'}

## category 별 데이터프레임

In [25]:
category_representation = pd.DataFrame(columns=sorted(unique_category), index=df['recipeID'])
print(category_representation.head(1))

         간 건강 골다공증 노화방지 머슬업 다이어트   변비 스트레스 해소 암 예방 위 건강 장 건강 저칼로리 다이어트 치매예방  \
recipeID                                                                      
0         NaN  NaN  NaN      NaN  NaN     NaN  NaN  NaN  NaN       NaN  NaN   

         피로회복 혈당조절 혈액순환 호흡기 건강  
recipeID                        
0         NaN  NaN  NaN    NaN  


In [26]:
for category in df['category']:
    # 해당하는 카테고리에는 1을 넣고 아니면 0을 넣음
    category_representation[category] = np.where(df['category'] == category, 2, None)

In [27]:
print(category_representation.tail(5))

          간 건강  골다공증  노화방지 머슬업 다이어트    변비 스트레스 해소  암 예방  위 건강  장 건강 저칼로리 다이어트  \
recipeID                                                                        
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
872       None  None  None     None  None    None  None  None  None      None   
873       None  None  None     None  None    None  None  None  None      None   

         치매예방  피로회복  혈당조절  혈액순환 호흡기 건강  
recipeID                                
872         2  None  None  None   None  
872         2  None  None  None   None  
872         2  None  None  None   None  
872         2  None  None  None   None  
873         2  None  None  None   None  


## keyword 별 데이터프레임

In [28]:
keyword_representation = pd.DataFrame(columns=sorted(unique_keyword), index=df['recipeID'])
print(keyword_representation.head(1))

            -  -말캉    . .꽁치캔   .맜  .무우  .알탕  .양뇸  2손다  3스픈  ...  흰우유   흰자  \
recipeID                                                    ...             
0         NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   

           흰콩  흰후추 흰후추가루    히   히트   힌트   힐링    힘  
recipeID                                           
0         NaN  NaN   NaN  NaN  NaN  NaN  NaN  NaN  

[1 rows x 4048 columns]


In [29]:
for index in tqdm(range(len(df['recipeID']))):
    keywords = df['all'][index]
    recipeID = df['recipeID'].iloc[index]
    for keyword in keywords:
        keyword_idf_value = keyword_idf.get(keyword.strip(), None) 
        if keyword_idf_value is not None:
            keyword_representation.at[recipeID, keyword] = keyword_idf_value

# keyword_representation.sort_index(inplace=True)

100%|██████████| 4025/4025 [00:07<00:00, 566.95it/s]


In [30]:
keyword_representation.head(5)

,-,-말캉,.,.꽁치캔,.맜,.무우,.알탕,.양뇸,2손다,3스픈,...,흰우유,흰자,흰콩,흰후추,흰후추가루,히,히트,힌트,힐링,힘
recipeID,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 카테고리 + 키워드

In [31]:
recipe_representation = pd.concat([category_representation, keyword_representation], axis=1).fillna(0)


In [32]:
recipe_representation = recipe_representation.drop_duplicates()
print(recipe_representation.shape)
print(recipe_representation.describe())

(871, 4063)
             간 건강        골다공증        노화방지    머슬업 다이어트          변비     스트레스 해소  \
count  871.000000  871.000000  871.000000  871.000000  871.000000  871.000000   
mean     0.137773    0.137773    0.133180    0.137773    0.130884    0.137773   
std      0.506813    0.506813    0.498908    0.506813    0.494893    0.506813   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
75%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
max      2.000000    2.000000    2.000000    2.000000    2.000000    2.000000   

             암 예방        위 건강        장 건강   저칼로리 다이어트  ...         흰우유  \
count  871.000000  871.000000  871.000000  871.000000  ...  871.000000   
mean     0.126292    0.137773    0.140069    0.137773  ...    0.005980   
std      0.486730    0.506813    0.5

In [33]:
def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])
    
    return result_df

In [34]:
cs_df = cos_sim_matrix(recipe_representation, recipe_representation)
cs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,861,862,863,864,865,866,867,868,869,870
recipeID,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.104482,0.099753,0.123909,0.115699,0.167870,0.093785,0.096656,0.212133,0.085687,...,0.000000,0.000173,0.000000,0.000000,0.000000,0.000105,0.000000,0.023696,0.014615,0.000000
1,0.104482,1.000000,0.081452,0.104113,0.097215,0.144960,0.079696,0.082396,0.180837,0.084474,...,0.000981,0.029849,0.000000,0.000000,0.000000,0.002090,0.003618,0.000000,0.001972,0.000000
2,0.099753,0.081452,1.000000,0.117808,0.204223,0.130867,0.072881,0.075367,0.165374,0.066800,...,0.000000,0.000009,0.000000,0.009902,0.000000,0.010413,0.000000,0.000000,0.000000,0.000050
3,0.123909,0.104113,0.117808,1.000000,0.115290,0.162558,0.090530,0.093617,0.205420,0.085384,...,0.000000,0.001542,0.000000,0.000037,0.000000,0.000929,0.002775,0.000000,0.002165,0.000030
4,0.115699,0.097215,0.204223,0.115290,1.000000,0.151788,0.084532,0.100553,0.193688,0.079829,...,0.000785,0.001976,0.012838,0.000000,0.000281,0.020280,0.038591,0.000000,0.002709,0.000029


In [35]:
cs_df.shape

(871, 871)

In [36]:
print(cs_df[1].sort_values(ascending=False))

recipeID
1           1.000000
18          0.219561
8           0.180837
750         0.174716
28          0.167877
              ...   
599         0.000000
597         0.000000
330         0.000000
595         0.000000
873         0.000000
Name: 1, Length: 871, dtype: float64


In [37]:
print(data.loc[1])
print(data.loc[18])
print(data.loc[8])
print(data.loc[750])

recipeID                                                           1
recipe_name                                           아삭아삭 연근장아찌 만들기
summary            연근은 비타민C와 식이섬유소가 풍부하고 빈혈예방에 탁월하고 코피가 자주나는 아이들에...
ingredient_name    [재료] 연근  [양념비율(200ml 계량컵 기준)] 간장  1 물  1 설탕  0...
full_step          연근은 감자칼로 껍질을 벗겨주었어요. 슥슥 0.5mm정도 잘라주었답니다. 찬물에 식...
category                                                   저칼로리 다이어트
image_link         https://recipe1.ezmember.co.kr/cache/recipe/20...
main_source                                                     [연근]
Name: 1, dtype: object
recipeID                                                          18
recipe_name                                               양파 장아찌 만들기
summary                 제철에 담아두면 일 년 내내 맛있게 드실 수가 있어서 참 좋은 양파장아찌입니다.
ingredient_name                              [재료] 양파  간장  물  설탕  식초 
full_step          양파는 적당한 크기로 썰어 주세요. 양파 담아줬어요. 내열강화유리라 뜨거운 절임물 ...
category                                                   저칼로리 다이어트
image_link 

## user 이웃기반

In [38]:
train_df, test_df = train_test_split(data_re, test_size=0.2, random_state=1234)

In [39]:
train_df.shape, test_df.shape

((2824, 8), (706, 8))

In [40]:
user_ids = sorted(set(train_df['user_id'].values))
recipe_ids = sorted(set(train_df['recipeID'].values))

print(f'유저 수: {len(user_ids)}, 레시피 수: {len(recipe_ids)}')


유저 수: 2646, 레시피 수: 355


In [41]:
sparse_matrix = pd.DataFrame(index=recipe_ids, columns=user_ids)
sparse_matrix

,0,1,2,6,7,9,10,11,13,14,...,3397,3398,3399,3400,3403,3404,3405,3406,3407,3408
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
train_df.head(10)

,recipe_name,user_id,posting_time,rating,review_content,category,recipeID,reviewID
1348,시래기표고버섯밥,17073180,2022-01-05 19:44:37,5,오.. 저녁완전 잘먹었어요♥︎,장 건강,305,238
3259,"견과류 듬뿍 넣은 쫄깃,달콤함 ""두부강정""♪",91691691,2022-05-19 12:18:32,5,두부를 꽤 오랫동안 구워야되더라구요! 쫄깃쫄깃 넘 맛있어요.\n신랑도 꿀맛이라고 극...,치매예방,828,2847
235,고등어무조림,19293848,2020-04-17 07:14:59,5,레시피 올려주셔서 엎드려 절하고 싶었습니다 이것은 정말 황금레시피.. 뭐 하나 가감...,머슬업 다이어트,84,324
671,고등어무조림,27724572,2023-02-10 20:59:56,5,정말 넘넘넘 맛있어요!!!! 다음에는 생물로도 해보고 싶어요 ㅎㅎㅎ,머슬업 다이어트,84,593
89,꼬시래기초무침 만들기,66056272,2022-05-22 21:29:39,5,"전에 한번 따라할 땐 넘 오래 데쳐서 망했는데, 이번엔 살짝 데치고 고추장이랑 고추...",저칼로리 다이어트,38,1947
2655,노밀가루 바나나 팬케이크,62090653,2018-08-29 17:41:08,5,오늘 두번 해먹었는데 믹서기에 갈아서 부의니 완전 핫케익 인데요? 뒤집기가 진짜 힘...,암 예방,713,1804
507,고등어무조림,51464809,2018-05-13 11:34:03,5,제가 다이어트중이라 ㅠㅠ 간도 안보고 레시피대로해줬어요 고등어 두개라 무도 두배 양...,머슬업 다이어트,84,1424
29,깨순볶음,44392416,2017-06-16 23:59:23,5,깨순에 양념을 묻히고 후라이팬에 익히니 간도 잘배어있고 맛있네요~,저칼로리 다이어트,20,1182
218,고등어무조림,90765257,2018-07-28 00:29:44,5,덕분에 맛있게 잘 먹었습니다!\n무는 적혀있는 것 보다 많이 넣어도 좋을 것 같아요!,머슬업 다이어트,84,2809
1130,달걀지단 치즈롤 샌드위치,60790108,2018-04-05 09:48:29,4,마요네즈하니 애들이 싱거운지 그위 딸기쨈을 얻어서 먹네요,위 건강,241,1751


In [43]:
train_df_grouped = train_df.groupby(['recipeID', 'user_id'])['rating'].mean().reset_index()
sparse_matrix = train_df_grouped.pivot(index='recipeID', columns='user_id', values='rating')
sparse_matrix.index.name = 'recipeID'
sparse_matrix

reviewID,0,1,2,6,7,9,10,11,13,14,...,3397,3398,3399,3400,3403,3404,3405,3406,3407,3408
recipeID,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
868,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
sparse_matrix.loc[779, 884]

5.0

In [46]:
def cossim_matrix(a, b):
    cossim_values = cosine_similarity(a.values, b.values)
    cossim_df = pd.DataFrame(data=cossim_values, columns = a.index.values, index=a.index)

    return cossim_df

In [47]:
item_sparse_matrix = sparse_matrix.fillna(0)
item_sparse_matrix

reviewID,0,1,2,6,7,9,10,11,13,14,...,3397,3398,3399,3400,3403,3404,3405,3406,3407,3408
recipeID,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
item_cossim_df = cossim_matrix(item_sparse_matrix, item_sparse_matrix)
item_cossim_df

,2,4,8,9,11,12,13,14,16,20,...,851,852,856,862,863,865,868,869,870,872
recipeID,,,,,,,,,,,,,,,,,,,,,
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.023062,0.000000,0.0
868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,0.000000,0.000000,0.0
869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.068461,0.023062,0.0,1.000000,0.068461,0.0


In [49]:
userId_grouped = train_df.groupby('user_id')

item_prediction_result_df = pd.DataFrame(index=userId_grouped.indices.keys(), columns=item_sparse_matrix.index)
item_prediction_result_df

recipeID,2,4,8,9,11,12,13,14,16,20,...,851,852,856,862,863,865,868,869,870,872
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
for user_id, group in tqdm(userId_grouped):
    # user 1의 movieId 를 가지고 item_cossim_df에서 유사도 추출
    user_sim = item_cossim_df.loc[group['recipeID']]
    user_rating = group['rating']

    # 사용자가 평가한 영화들의 유사도 총합을 계산
    sim_sum = user_sim.sum(axis=0)
    
    pred_ratings = np.matmul(user_sim.T.to_numpy(), user_rating) / (sim_sum+1)
    item_prediction_result_df.loc[user_id] = pred_ratings

  0%|          | 0/2646 [00:00<?, ?it/s]

100%|██████████| 2646/2646 [00:03<00:00, 779.92it/s]


In [51]:
item_prediction_result_df.head(10)

recipeID,2,4,8,9,11,12,13,14,16,20,...,851,852,856,862,863,865,868,869,870,872
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200301,...,0.0,0.0,0.0,0.0,0.0,0.141576,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.228072,0.0,0.194284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054477,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.146191,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
print(item_prediction_result_df.shape)

(2646, 355)


In [53]:
def evaluate(test_df, prediction_result_df):
  groups_with_recipe_ids = test_df.groupby(by='recipeID')
  groups_with_user_ids = test_df.groupby(by='user_id')
  
  # 트레인 테스트 df에도 있는 무비 ID와 유저 아이디 가지고 오기
  intersection_recipe_ids = sorted({index for index in prediction_result_df.columns}.intersection(set(groups_with_recipe_ids.indices.keys())))
  # intersection_recipe_ids = sorted(set(prediction_result_df.columns).intersection(set(groups_with_recipe_ids.indices.keys())))
  intersection_user_ids = sorted({index for index in prediction_result_df.index}.intersection(set(groups_with_user_ids.indices.keys())))

  print(len(intersection_recipe_ids))
  print(len(intersection_user_ids))
  
  compressed_prediction_df = prediction_result_df.loc[intersection_user_ids][intersection_recipe_ids]
  
  grouped = test_df.groupby(by='user_id')
  result_df = pd.DataFrame(columns=['rmse'])
  
  for userId, group in tqdm(grouped):
      if userId in intersection_user_ids:
          pred_ratings = compressed_prediction_df.loc[userId][compressed_prediction_df.loc[userId].index.\
              intersection(group['recipeID'].values)]
          pred_ratings = pred_ratings.to_frame(name='rating').reset_index().\
              rename(columns={'index':'recipeID','rating':'pred_rating'})
          actual_ratings = group[['rating', 'recipeID']].rename(columns={'rating':'actual_rating'})
          
          final_df = pd.merge(actual_ratings, pred_ratings, how='inner', on=['recipeID'])
          final_df = final_df.round(4)
          
  return final_df

In [54]:
evaluate(test_df, item_prediction_result_df)

163
70


 18%|█▊        | 124/695 [00:00<00:00, 1213.73it/s]

100%|██████████| 695/695 [00:00<00:00, 1709.03it/s]


,actual_rating,recipeID,pred_rating
0,5,245,0.11292


In [55]:
result_df = evaluate(test_df, item_prediction_result_df)
print(result_df)
print(f"RMSE: {sqrt(mean_squared_error(result_df['actual_rating'].values, result_df['pred_rating'].values))}")

163
70


100%|██████████| 695/695 [00:00<00:00, 2742.60it/s]

   actual_rating  recipeID pred_rating
0              5       245     0.11292
RMSE: 4.887079894394488
